In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import seaborn as sns

import pytelligence as pt

%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 2000

sns.set_theme()

/home/george/Desktop/pycarrot/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# bc = load_breast_cancer()
# X = pd.DataFrame(bc.data, columns=bc.feature_names)
# y = pd.Series(bc.target, name="class")
# df = pd.concat([X, y], axis=1)
# df.head()

In [3]:
df = pd.read_csv(
    "./data/breast_cancer_cat/breast-cancer.data",
    names=[
        "class",
        "age",
        "menopause",
        "tumor-size",
        "inv-nodes",
        "node-caps",
        "deg-malig",
        "breast",
        "breast-quad",
        "irradiat",
    ],
)
df.head()

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [4]:
# pt.feat_analysis.get_distribution(df)

In [5]:
# sns.jointplot(data=df, x="deg-malig", y="breast", hue="class")

## Training

In [6]:
df.columns

Index(['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
       'deg-malig', 'breast', 'breast-quad', 'irradiat'],
      dtype='object')

In [7]:
setup, X_sample, y_sample = pt.modelling.prepare_data(
    train_data=df,
    config_path="./config_bc_cat.yml",
)

[I 2022-07-14 23:37:16] %%% PREPARING DATA
[I 2022-07-14 23:37:16] Read ./config_bc_cat.yml: 
 {'modelling': {'target_clf': 'class', 'numeric_cols': ['deg-malig'], 'categorical_cols': ['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'breast', 'breast-quad', 'irradiat'], 'feature_scaling': False}}
[I 2022-07-14 23:37:16] Created preprocessing pipeline with following steps: ['ohe']
[I 2022-07-14 23:37:16] Applied preprocessing transformations.
[I 2022-07-14 23:37:16] Encoded target variable using classes: [(0, 'no-recurrence-events'), (1, 'recurrence-events')]


In [8]:
X_sample

,deg-malig,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,menopause_lt40,menopause_premeno,tumor-size_0-4,tumor-size_10-14,tumor-size_15-19,tumor-size_20-24,tumor-size_25-29,tumor-size_30-34,tumor-size_35-39,tumor-size_40-44,tumor-size_45-49,tumor-size_5-9,tumor-size_50-54,inv-nodes_0-2,inv-nodes_12-14,inv-nodes_15-17,inv-nodes_24-26,inv-nodes_3-5,inv-nodes_6-8,inv-nodes_9-11,node-caps_?,node-caps_no,node-caps_yes,breast_left,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no,irradiat_yes
0,3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
1,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0
2,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
3,2,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0
4,2,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0


In [8]:
compare_df, algo_list, model_list = pt.modelling.compare_algorithms(
    setup=setup,
    include=[
        "lr",
        # "dt",
        # "rf",
        # "ridge",
        # "perceptron",
        # "passive-aggressive",
        # "extratree",
        # "extratrees",
        # "knn",
        "nb",
        # "linearsvc",
        # "rbfsvc",
    ],
    sort="f1",
    return_models=True,
)
compare_df

[I 2022-07-14 23:41:02] %%% COMPARING ALGORITHMS
[W 2022-07-14 23:41:02] The algorithms ['lr'] work suboptimally without scaled features. Consider turning it on within the config.
[I 2022-07-14 23:41:02] Begin evaluating lr
[I 2022-07-14 23:41:13] Begin evaluating nb


,algorithm,accuracy,precision,recall,f1,roc_auc,Fit time (s)
0,nb,0.549,0.447,0.835,0.552,0.659,0.067
1,lr,0.678,0.527,0.329,0.338,0.653,0.164


In [10]:
le = setup.y_clf_encoder
preds = model_list[-1].predict(X_sample)
le.inverse_transform(preds)

array(['recurrence-events', 'no-recurrence-events',
       'no-recurrence-events', 'no-recurrence-events',
       'no-recurrence-events'], dtype=object)

In [11]:
print(*[(i, class_) for i, class_ in enumerate(le.classes_)])

(0, 'no-recurrence-events') (1, 'recurrence-events')


In [13]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [12]:
compare_df_tune, model_list = (
        pt.modelling.tune_hyperparams(
               setup=setup,
               include=["lr", "nb"],
               optimize="f1",
               n_trials=5,
               return_models=True,
        )
)
compare_df_tune

[I 2022-07-14 23:54:41] %%% TUNING HYPERPARAMETERS
[W 2022-07-14 23:54:41] The algorithms ['lr'] work suboptimally without scaled features. Consider turning it on within the config.
[I 2022-07-14 23:54:41] Algorithms selected for tuning: ['lr', 'nb']
[I 2022-07-14 23:54:41] Metric to optimize for: f1
[I 2022-07-14 23:54:41] Trials per algorithm: 5
[I 2022-07-14 23:54:41,690] A new study created in memory with name: study_lr
[I 2022-07-14 23:54:42,828] Trial 0 finished with value: 0.38694824016563145 and parameters: {'C': 4.169451287796391, 'l1_ratio': 0.6173659271431332}. Best is trial 0 with value: 0.38694824016563145.
[I 2022-07-14 23:54:44,599] Trial 1 finished with value: 0.3721659499920369 and parameters: {'C': 13.816842393861599, 'l1_ratio': 0.2644421571626768}. Best is trial 0 with value: 0.38694824016563145.
[I 2022-07-14 23:54:44,768] Trial 2 finished with value: 0.0 and parameters: {'C': 0.000400316078767674, 'l1_ratio': 0.05123957859780859}. Best is trial 0 with value: 0.386

,algorithm,metric,hyperparams
0,nb,0.558979,"{'priors': None, 'var_smoothing': 2.9165556862..."
1,lr,0.386948,"{'C': 4.169451287796391, 'class_weight': None,..."


In [14]:
best_feature_list = pt.modelling.reduce_feature_space(
    setup=setup,
    algorithm= "nb",
    metric="f1",
    reference_metric=compare_df_tune.iloc[0]["metric"],
    acceptable_loss=0.95,
    hyperparams=compare_df_tune.iloc[0]["hyperparams"]
)

[I 2022-07-15 00:09:53] %%% REDUCING FEATURE SPACE
[I 2022-07-15 00:09:53] Algorithm selected for feature space reduction: nb
[I 2022-07-15 00:09:53] Metric to optimize for: f1
[I 2022-07-15 00:09:53] Minimum acceptable metric: 0.531 (0.95 * reference metric 0.559)
[I 2022-07-15 00:10:04] Current metric: 0.571, removing worst feature: breast-quad_right_up
[I 2022-07-15 00:10:04] Updating reference metric...
[I 2022-07-15 00:10:07] Current metric: 0.577, removing worst feature: tumor-size_50-54
[I 2022-07-15 00:10:07] Updating reference metric...
[I 2022-07-15 00:10:10] Current metric: 0.582, removing worst feature: age_70-79
[I 2022-07-15 00:10:10] Updating reference metric...
[I 2022-07-15 00:10:13] Current metric: 0.584, removing worst feature: breast-quad_central
[I 2022-07-15 00:10:13] Updating reference metric...
[I 2022-07-15 00:10:16] Current metric: 0.585, removing worst feature: tumor-size_30-34
[I 2022-07-15 00:10:16] Updating reference metric...
[I 2022-07-15 00:10:19] Curre

In [15]:
compare_df_final, model_list = (
        pt.modelling.tune_hyperparams(
               setup=setup,
               include=["nb"],
               optimize="f1",
               n_trials=50,
               feature_list=best_feature_list,
               return_models=True,
        )
)
compare_df_final

[I 2022-07-15 00:13:05] %%% TUNING HYPERPARAMETERS
[W 2022-07-15 00:13:05] The algorithms [] work suboptimally without scaled features. Consider turning it on within the config.
[I 2022-07-15 00:13:05] Algorithms selected for tuning: ['nb']
[I 2022-07-15 00:13:05] Metric to optimize for: f1
[I 2022-07-15 00:13:05] Trials per algorithm: 50
[I 2022-07-15 00:13:05,473] A new study created in memory with name: study_nb
[I 2022-07-15 00:13:05,657] Trial 0 finished with value: 0.6137072431412054 and parameters: {'var_smoothing': 7.066737492969319e-10}. Best is trial 0 with value: 0.6137072431412054.
[I 2022-07-15 00:13:05,865] Trial 1 finished with value: 0.5161464131665823 and parameters: {'var_smoothing': 0.01381017150659007}. Best is trial 0 with value: 0.6137072431412054.
[I 2022-07-15 00:13:06,086] Trial 2 finished with value: 0.6137072431412054 and parameters: {'var_smoothing': 2.4561982076231187e-10}. Best is trial 0 with value: 0.6137072431412054.
[I 2022-07-15 00:13:06,286] Trial 3 

,algorithm,metric,hyperparams
0,nb,0.613707,"{'priors': None, 'var_smoothing': 7.0667374929..."


In [37]:
setup.prep_pipe

Pipeline(steps=[('ohe', OHE())])

In [36]:
pt.modelling.export_model(
        setup=setup,
        model=model_list[0],
        target_dir="./",
    )